Tools used in Python:

    pandasql library: SQLite dialect for SQL
    requests library: to query the REST API of US Census
    urllib library: to query the REST API of US Census
    pandas library: to work with dataframes

In [2]:
import pandas as pd
import requests

In [3]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

import json

# Sign up for a free Census API key here: https://api.census.gov/data/key_signup.html
# Census API Guide: https://www.census.gov/data/developers/guidance.html

with open(r'Census_Data_API_Key.json') as fp:
    census_api_key = json.load(fp)
census_api_key = census_api_key['census_api_key']
census_api_key

url = "https://api.census.gov/data/2019/pep/charage?get=NAME,POP&for=state:*&key={0}".format(census_api_key)
response = requests.request("GET", url)
response.text
temp_df = json_to_dataframe(response)
states_df = pd.read_csv(r'us_states_df.csv')
state_codes_df = temp_df.merge(states_df, left_on='NAME', right_on='STATE(TERRITORY)').drop(columns=['POP','NAME'])
state_codes_df.columns = ['FIPS State Numeric Code', 'Name', 'Official USPS Code']
state_codes_df

,FIPS State Numeric Code,Name,Official USPS Code
0,01,Alabama,AL
1,02,Alaska,AK
2,04,Arizona,AZ
3,05,Arkansas,AR
4,06,California,CA
5,09,Connecticut,CT
6,12,Florida,FL
7,13,Georgia,GA
8,17,Illinois,IL
9,23,Maine,ME


In [4]:
from urllib.request import urlopen

temp_list1 = []
for index, row in state_codes_df.iterrows():
    temp_list = []
    url = f"https://www2.census.gov/geo/docs/reference/codes/files/st{row['FIPS State Numeric Code']}_{row['Official USPS Code'].lower()}_cou.txt"
    response = urlopen(url)
    data = response.read()
    temp_list.append(data.decode("utf-8").split('\r\n'))
    temp_list1.append(temp_list)

print(len(temp_list1))

# first save element in index 1 in list then pop element in index 1 from list
list_different_from_others = temp_list1[1][0][0].split('\n')[:-1]
temp_list1.pop(1)

# then find len(temp_list1)
temp_list2 = []
for j in range(len(temp_list1)):
    for i in temp_list1[j][0]:
        temp_list2.append(i)
for i in list_different_from_others:
    temp_list2.append(i)

state_county_df = pd.DataFrame(temp_list2, columns=['col1'])
state_county_df = state_county_df['col1'].str.split(',', expand=True)
state_county_df.columns = ['STATE', 'STATEFP', 'COUNTYFP', 'COUNTYNAME', 'CLASSFP']
state_county_df

52


,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1
...,...,...,...,...,...
3216,AK,02,261,Valdez-Cordova Census Area,H5
3217,AK,02,270,Wade Hampton Census Area,H5
3218,AK,02,275,Wrangell City and Borough,H1
3219,AK,02,282,Yakutat City and Borough,H1


# 2017 abscs dataset

In [5]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

# URL for all datasets: https://api.census.gov/data.html
# 2017 abscbo dataset is not chosen because the number of firms column (FIRMPDEMP) is not in it. Thus, 2017 abscs which has that column is chosen. 
# For detailed information on the variable list, abbreviation meanings, and more, refer to the Census API Documentation for each specific dataset
# Example for sample dataset: https://api.census.gov/data/2017/abscs/variables.html
# Sample documentation: https://www.census.gov/data/developers/data-sets/abs.html

url = f"https://api.census.gov/data/2017/abscbo?get=NAME,GEO_ID,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F&for=county:*&in=state:*&NAICS2017=00&key={census_api_key}"
response = requests.request("GET", url)

census_county_business_df = json_to_dataframe(response)
census_county_business_df

,NAME,GEO_ID,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,...,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F,NAICS2017,state,county
0,"Warren County, Tennessee",0500000US47177,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,48.6,None,21.4,None,8.9,None,00,47,177
1,"Washington County, Tennessee",0500000US47179,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,69.0,None,8.2,None,9.2,None,00,47,179
2,"Weakley County, Tennessee",0500000US47183,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,62.7,None,12.9,None,8.9,None,00,47,183
3,"Benton County, Tennessee",0500000US47005,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,52.4,None,31.4,None,12.2,None,00,47,005
4,"Bledsoe County, Tennessee",0500000US47007,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,S,0.0,S,0.0,S,0.0,S,00,47,007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,"Martin County, Minnesota",0500000US27091,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,44.3,None,28.2,None,12.6,None,00,27,091
3107,"Mower County, Minnesota",0500000US27099,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,45.3,None,29.4,None,10.0,None,00,27,099
3108,"Nobles County, Minnesota",0500000US27105,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,59.2,None,26.6,None,11.9,None,00,27,105
3109,"Pine County, Minnesota",0500000US27115,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,001,...,None,65.5,None,19.0,None,7.2,None,00,27,115


In [6]:
print("There is census data for",census_county_business_df['NAME'].nunique(), "counties")

There is census data for 3111 counties


#### Note that pandasql uses SQLite dialect

In [7]:
url = f"https://api.census.gov/data/2017/abscs?get=NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP&for=county:*&in=state:*&NAICS2017=00&key={census_api_key}"
response = requests.request("GET", url)
census_df = json_to_dataframe(response)
print(census_df.shape)
census_df['FIRMPDEMP'] = census_df['FIRMPDEMP'].astype(int)

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
query = """SELECT NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP,NAICS2017,state,county, 
max(FIRMPDEMP) OVER(PARTITION BY NAME, GEO_ID) as total_businesses
FROM census_df;"""
census_df = pysqldf(query)
census_df

### Pandas Aliter for same above SQL statement
# census_df['total_businesses'] = census_df.groupby(by=['NAME', 'GEO_ID'])['FIRMPDEMP'].transform(np.max)
# census_df.drop_duplicates()

print("There is census data for",census_df['NAME'].nunique(), "counties")
census_df = census_df[census_df['total_businesses']!=0]
print("Filtering out counties where there are no records of businesses (maybe a lack of surveys); census_df remaining counties is", census_df['NAME'].nunique())

# Female owned OR Minority owned OR Vet owned businesses
census_df_filtered = census_df[(census_df['SEX']=='002')|(census_df['RACE_GROUP']=='90')|(census_df['VET_GROUP']=='002')].copy(deep=True) 
print("After filtering census_df for specific disadvantaged groups that we have data for, we have",census_df_filtered.shape[0], "observations and remaining number of counties is", census_df_filtered['NAME'].nunique())

(51730, 11)
There is census data for 3140 counties
Filtering out counties where there are no records of businesses (maybe a lack of surveys); census_df remaining counties is 3058
After filtering census_df for specific disadvantaged groups that we have data for, we have 8766 observations and remaining number of counties is 3057


In [8]:
census_df[census_df['total_businesses']==0] # Checking number of observations with zero total_businesses

,NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP,NAICS2017,state,county,total_businesses
